**MACD-Bollinger Bands Combined Strategy**

The aim is to potentially reduce false signals by requiring stronger alignment between indicators for signal generation.
The strategy introduces a more conservative approach, generating buy signals only when both MACD and Bollinger Bands strongly indicate a bullish trend. Sell signals are generated when both indicators signal a bearish trend, aiming for more precise and aligned signals.

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

**Stock 1:APPLE**

In [ ]:
data=yf.download('AAPL')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099319,469033600
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094137,175884800
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087228,105728000
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089387,86441600
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091978,73449600
...,...,...,...,...,...,...
2023-12-18,196.089996,196.630005,194.389999,195.889999,195.889999,55751900
2023-12-19,196.160004,196.949997,195.889999,196.940002,196.940002,40714100
2023-12-20,196.899994,197.679993,194.830002,194.830002,194.830002,52242800


In [ ]:
# MACD parameters for signal generation
short_ema_period = 12
long_ema_period = 26
signal_ema_period = 9

# Calculate MACD
short_ema = data['Close'].ewm(span=short_ema_period, adjust=False).mean()
long_ema = data['Close'].ewm(span=long_ema_period, adjust=False).mean()
macd_line = short_ema - long_ema
signal_line = macd_line.ewm(span=signal_ema_period, adjust=False).mean()
signal_line

Date
1980-12-12    0.000000
1980-12-15   -0.000107
1980-12-16   -0.000417
1980-12-17   -0.000793
1980-12-18   -0.001136
                ...   
2023-12-18    3.573223
2023-12-19    3.559025
2023-12-20    3.492683
2023-12-21    3.386783
2023-12-22    3.236689
Name: Close, Length: 10849, dtype: float64

In [ ]:
# Bollinger Bands parameters
period = 20
std_dev_multiplier = 2.0

# Calculate Bollinger Bands
std_dev = data['Close'].rolling(window=period).std()
data['SMA'] = data['Close'].rolling(window=period).mean()
data['Upper Band'] = data['SMA'] + std_dev_multiplier * std_dev
data['Lower Band'] = data['SMA'] - std_dev_multiplier * std_dev
data['Upper Band']

Date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16           NaN
1980-12-17           NaN
1980-12-18           NaN
                 ...    
2023-12-18    198.770131
2023-12-19    199.278227
2023-12-20    199.418889
2023-12-21    199.539782
2023-12-22    199.490535
Name: Upper Band, Length: 10849, dtype: float64

In [ ]:
data['Lower Band']

Date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16           NaN
1980-12-17           NaN
1980-12-18           NaN
                 ...    
2023-12-18    186.898869
2023-12-19    186.939774
2023-12-20    187.218112
2023-12-21    187.434219
2023-12-22    187.846466
Name: Lower Band, Length: 10849, dtype: float64

In [ ]:
# Signal Generation based on MACD and Bollinger Bands
signals = []

for i in range(len(data)):
    if macd_line.iloc[i] > signal_line.iloc[i] and data['Close'].iloc[i] > data['Upper Band'].iloc[i]:
        signals.append("Buy")
    elif macd_line.iloc[i] < signal_line.iloc[i] and data['Close'].iloc[i] < data['Lower Band'].iloc[i]:
        signals.append("Sell")
    else:
        signals.append("Hold")

data['Signals'] = signals
data['Signals']

Date
1980-12-12    Hold
1980-12-15    Hold
1980-12-16    Hold
1980-12-17    Hold
1980-12-18    Hold
              ... 
2023-12-18    Hold
2023-12-19    Hold
2023-12-20    Hold
2023-12-21    Hold
2023-12-22    Hold
Name: Signals, Length: 10849, dtype: object

In [ ]:
portfolio_value = 100000  # Initial portfolio value
cash = portfolio_value
stocks = 0
position = "Neutral"
transaction_log = []

for i in range(len(data)):
    if data['Signals'].iloc[i] == "Buy" and position != "Buy":
        stocks += cash / data['Close'].iloc[i]
        cash = 0
        position = "Buy"
        transaction_log.append(("Buy", data.index[i], data['Close'].iloc[i]))

    elif data['Signals'].iloc[i] == "Sell" and position != "Sell":
        cash += stocks * data['Close'].iloc[i]
        stocks = 0
        position = "Sell"
        transaction_log.append(("Sell", data.index[i], data['Close'].iloc[i]))

final_portfolio_value = cash + (stocks * data['Close'].iloc[-1])
final_portfolio_value

276035751.97048444

**Stock 2:GOOGLE**

In [ ]:
data=yf.download('GOOGL')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2004-08-19,2.502503,2.604104,2.401401,2.511011,2.511011,893181924
2004-08-20,2.527778,2.729730,2.515015,2.710460,2.710460,456686856
2004-08-23,2.771522,2.839840,2.728979,2.737738,2.737738,365122512
2004-08-24,2.783784,2.792793,2.591842,2.624374,2.624374,304946748
2004-08-25,2.626627,2.702703,2.599600,2.652653,2.652653,183772044
...,...,...,...,...,...,...
2023-12-18,132.630005,137.149994,132.429993,135.800003,135.800003,32258000
2023-12-19,136.839996,137.470001,136.080002,136.649994,136.649994,25476800
2023-12-20,138.970001,141.699997,138.070007,138.339996,138.339996,49107200


In [ ]:
# MACD parameters for signal generation
short_ema_period = 12
long_ema_period = 26
signal_ema_period = 9

# Calculate MACD
short_ema = data['Close'].ewm(span=short_ema_period, adjust=False).mean()
long_ema = data['Close'].ewm(span=long_ema_period, adjust=False).mean()
macd_line = short_ema - long_ema
signal_line = macd_line.ewm(span=signal_ema_period, adjust=False).mean()
signal_line

Date
2004-08-19    0.000000
2004-08-20    0.003182
2004-08-23    0.008620
2004-08-24    0.013358
2004-08-25    0.017830
                ...   
2023-12-18    0.039065
2023-12-19    0.092472
2023-12-20    0.200920
2023-12-21    0.370908
2023-12-22    0.586784
Name: Close, Length: 4871, dtype: float64

In [ ]:
# Bollinger Bands parameters
period = 20
std_dev_multiplier = 2.0

# Calculate Bollinger Bands
std_dev = data['Close'].rolling(window=period).std()
data['SMA'] = data['Close'].rolling(window=period).mean()
data['Upper Band'] = data['SMA'] + std_dev_multiplier * std_dev
data['Lower Band'] = data['SMA'] - std_dev_multiplier * std_dev
data['Upper Band']

Date
2004-08-19           NaN
2004-08-20           NaN
2004-08-23           NaN
2004-08-24           NaN
2004-08-25           NaN
                 ...    
2023-12-18    139.462470
2023-12-19    139.518953
2023-12-20    139.771046
2023-12-21    140.235135
2023-12-22    141.223106
Name: Upper Band, Length: 4871, dtype: float64

In [ ]:
data['Lower Band']

Date
2004-08-19           NaN
2004-08-20           NaN
2004-08-23           NaN
2004-08-24           NaN
2004-08-25           NaN
                 ...    
2023-12-18    128.768535
2023-12-19    128.752051
2023-12-20    128.636958
2023-12-21    128.365867
2023-12-22    127.857897
Name: Lower Band, Length: 4871, dtype: float64

In [ ]:
# Signal Generation based on MACD and Bollinger Bands
signals = []

for i in range(len(data)):
    if macd_line.iloc[i] > signal_line.iloc[i] and data['Close'].iloc[i] > data['Upper Band'].iloc[i]:
        signals.append("Buy")
    elif macd_line.iloc[i] < signal_line.iloc[i] and data['Close'].iloc[i] < data['Lower Band'].iloc[i]:
        signals.append("Sell")
    else:
        signals.append("Hold")

data['Signals'] = signals
data['Signals']

Date
2004-08-19    Hold
2004-08-20    Hold
2004-08-23    Hold
2004-08-24    Hold
2004-08-25    Hold
              ... 
2023-12-18    Hold
2023-12-19    Hold
2023-12-20    Hold
2023-12-21     Buy
2023-12-22     Buy
Name: Signals, Length: 4871, dtype: object

In [ ]:
portfolio_value = 100000  # Initial portfolio value
cash = portfolio_value
stocks = 0
position = "Neutral"
transaction_log = []

for i in range(len(data)):
    if data['Signals'].iloc[i] == "Buy" and position != "Buy":
        stocks += cash / data['Close'].iloc[i]
        cash = 0
        position = "Buy"
        transaction_log.append(("Buy", data.index[i], data['Close'].iloc[i]))

    elif data['Signals'].iloc[i] == "Sell" and position != "Sell":
        cash += stocks * data['Close'].iloc[i]
        stocks = 0
        position = "Sell"
        transaction_log.append(("Sell", data.index[i], data['Close'].iloc[i]))

final_portfolio_value = cash + (stocks * data['Close'].iloc[-1])
final_portfolio_value

608154.721560528

**Stock 3: DELL**

In [ ]:
data=yf.download('DELL')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-08-17,12.207997,12.348319,12.067676,12.067676,11.438140,271519
2016-08-18,12.348319,12.348319,11.927354,11.997515,11.371639,1767366
2016-08-19,11.983482,12.207997,11.969450,12.207997,11.571141,4735900
2016-08-22,12.067676,12.797349,11.955418,12.250094,11.611042,2245909
2016-08-23,12.278158,12.881542,12.207997,12.628963,11.970147,1483020
...,...,...,...,...,...,...
2023-12-18,71.620003,73.680000,71.050003,73.370003,73.370003,4810300
2023-12-19,73.699997,74.010002,72.860001,73.529999,73.529999,2684000
2023-12-20,73.540001,74.279999,72.309998,72.419998,72.419998,4336900


In [ ]:
# MACD parameters for signal generation
short_ema_period = 12
long_ema_period = 26
signal_ema_period = 9

# Calculate MACD
short_ema = data['Close'].ewm(span=short_ema_period, adjust=False).mean()
long_ema = data['Close'].ewm(span=long_ema_period, adjust=False).mean()
macd_line = short_ema - long_ema
signal_line = macd_line.ewm(span=signal_ema_period, adjust=False).mean()
signal_line

Date
2016-08-17    0.000000
2016-08-18   -0.001119
2016-08-19    0.000479
2016-08-22    0.004368
2016-08-23    0.015522
                ...   
2023-12-18    0.036311
2023-12-19    0.080774
2023-12-20    0.125777
2023-12-21    0.214740
2023-12-22    0.332326
Name: Close, Length: 1851, dtype: float64

In [ ]:
# Bollinger Bands parameters
period = 20
std_dev_multiplier = 2.0

# Calculate Bollinger Bands
std_dev = data['Close'].rolling(window=period).std()
data['SMA'] = data['Close'].rolling(window=period).mean()
data['Upper Band'] = data['SMA'] + std_dev_multiplier * std_dev
data['Lower Band'] = data['SMA'] - std_dev_multiplier * std_dev
data['Upper Band']

Date
2016-08-17          NaN
2016-08-18          NaN
2016-08-19          NaN
2016-08-22          NaN
2016-08-23          NaN
                ...    
2023-12-18    77.028054
2023-12-19    77.047424
2023-12-20    76.929979
2023-12-21    77.044715
2023-12-22    77.274426
Name: Upper Band, Length: 1851, dtype: float64

In [ ]:
data['Lower Band']

Date
2016-08-17          NaN
2016-08-18          NaN
2016-08-19          NaN
2016-08-22          NaN
2016-08-23          NaN
                ...    
2023-12-18    66.718946
2023-12-19    66.716576
2023-12-20    66.714020
2023-12-21    66.663284
2023-12-22    66.563573
Name: Lower Band, Length: 1851, dtype: float64

In [ ]:
# Signal Generation based on MACD and Bollinger Bands
signals = []

for i in range(len(data)):
    if macd_line.iloc[i] > signal_line.iloc[i] and data['Close'].iloc[i] > data['Upper Band'].iloc[i]:
        signals.append("Buy")
    elif macd_line.iloc[i] < signal_line.iloc[i] and data['Close'].iloc[i] < data['Lower Band'].iloc[i]:
        signals.append("Sell")
    else:
       signals.append("Hold")

data['Signals'] = signals
data['Signals']

Date
2016-08-17    Hold
2016-08-18    Hold
2016-08-19    Hold
2016-08-22    Hold
2016-08-23    Hold
              ... 
2023-12-18    Hold
2023-12-19    Hold
2023-12-20    Hold
2023-12-21    Hold
2023-12-22    Hold
Name: Signals, Length: 1851, dtype: object

In [ ]:
# Strategy Execution Logic
portfolio_value = 100000  # Initial portfolio value
cash = portfolio_value
stocks = 0
position = "Neutral"
transaction_log = []

for i in range(len(data)):
    if data['Signals'].iloc[i] == "Buy" and position != "Buy":
        stocks += cash / data['Close'].iloc[i]
        cash = 0
        position = "Buy"
        transaction_log.append(("Buy", data.index[i], data['Close'].iloc[i]))

    elif data['Signals'].iloc[i] == "Sell" and position != "Sell":
        cash += stocks * data['Close'].iloc[i]
        stocks = 0
        position = "Sell"
        transaction_log.append(("Sell", data.index[i], data['Close'].iloc[i]))

final_portfolio_value = cash + (stocks * data['Close'].iloc[-1])
final_portfolio_value

189991.79042177158

**Backtesting on Microsoft Stock**

In [ ]:
import yfinance as yf

ticker = "MSFT"
start_date = "2018-01-01"
end_date = "2023-01-01"

# Fetch historical stock data
stock_data = yf.download(ticker, start=start_date, end=end_date)

# Display the fetched data
stock_data.head()


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,86.129997,86.309998,85.500000,85.949997,80.228996,22483800
2018-01-03,86.059998,86.510002,85.970001,86.349998,80.602379,26061400
2018-01-04,86.589996,87.660004,86.570000,87.110001,81.311806,21912000
2018-01-05,87.660004,88.410004,87.430000,88.190002,82.319923,23407100
2018-01-08,88.199997,88.580002,87.599998,88.279999,82.403908,22113000


In [ ]:
# MACD parameters for signal generation
short_ema_period = 12
long_ema_period = 26
signal_ema_period = 9

# Calculate MACD
short_ema = stock_data['Close'].ewm(span=short_ema_period, adjust=False).mean()
long_ema = stock_data['Close'].ewm(span=long_ema_period, adjust=False).mean()
macd_line = short_ema - long_ema
signal_line = macd_line.ewm(span=signal_ema_period, adjust=False).mean()
signal_line

Date
2018-01-02    0.000000
2018-01-03    0.006382
2018-01-04    0.028540
2018-01-05    0.076590
2018-01-08    0.139612
                ...   
2022-12-23    1.460951
2022-12-27    1.003037
2022-12-28    0.511894
2022-12-29    0.127889
2022-12-30   -0.188112
Name: Close, Length: 1259, dtype: float64

In [ ]:
# Bollinger Bands parameters
period = 20
std_dev_multiplier = 2.0

# Calculate Bollinger Bands
std_dev = stock_data['Close'].rolling(window=period).std()
stock_data['SMA'] = stock_data['Close'].rolling(window=period).mean()
stock_data['Upper Band'] = stock_data['SMA'] + std_dev_multiplier * std_dev
stock_data['Lower Band'] = stock_data['SMA'] - std_dev_multiplier * std_dev
stock_data['Upper Band']

Date
2018-01-02           NaN
2018-01-03           NaN
2018-01-04           NaN
2018-01-05           NaN
2018-01-08           NaN
                 ...    
2022-12-23    259.732099
2022-12-27    260.096177
2022-12-28    260.646931
2022-12-29    259.555356
2022-12-30    258.416996
Name: Upper Band, Length: 1259, dtype: float64

In [ ]:
stock_data['Lower Band']

Date
2018-01-02           NaN
2018-01-03           NaN
2018-01-04           NaN
2018-01-05           NaN
2018-01-08           NaN
                 ...    
2022-12-23    234.607899
2022-12-27    233.763823
2022-12-28    232.633068
2022-12-29    232.311642
2022-12-30    231.963003
Name: Lower Band, Length: 1259, dtype: float64

In [ ]:
# Signal Generation based on MACD and Bollinger Bands
signals = []

for i in range(len(stock_data)):
    if macd_line.iloc[i] > signal_line.iloc[i] and stock_data['Close'].iloc[i] > stock_data['Upper Band'].iloc[i]:
        signals.append("Buy")
    elif macd_line.iloc[i] < signal_line.iloc[i] and stock_data['Close'].iloc[i] < stock_data['Lower Band'].iloc[i]:
        signals.append("Sell")
    else:
        signals.append("Hold")

stock_data['Signals'] = signals
stock_data['Signals']

Date
2018-01-02    Hold
2018-01-03    Hold
2018-01-04    Hold
2018-01-05    Hold
2018-01-08    Hold
              ... 
2022-12-23    Hold
2022-12-27    Hold
2022-12-28    Hold
2022-12-29    Hold
2022-12-30    Hold
Name: Signals, Length: 1259, dtype: object

In [182]:
# Strategy Execution Logic
portfolio_value = 100000  # Initial portfolio value
cash = portfolio_value
stocks = 0
position = "Neutral"
transaction_log = []

for i in range(len(stock_data)):
    if stock_data['Signals'].iloc[i] == "Buy" and position != "Buy":
        stocks += cash / stock_data['Close'].iloc[i]
        cash = 0
        position = "Buy"
        transaction_log.append(("Buy", stock_data.index[i], stock_data['Close'].iloc[i]))

    elif stock_data['Signals'].iloc[i] == "Sell" and position != "Sell":
        cash += stocks * stock_data['Close'].iloc[i]
        stocks = 0
        position = "Sell"
        transaction_log.append(("Sell", stock_data.index[i], stock_data['Close'].iloc[i]))

final_portfolio_value = cash + (stocks * stock_data['Close'].iloc[-1])
final_portfolio_value


148839.94804177003

In [ ]:
total_returns = ((final_portfolio_value - portfolio_value) / portfolio_value) * 100
print(f"Final Total Returns: {total_returns:.2f}%")


Final Total Returns: 48.84%


In [ ]:
# Calculate buy-and-hold returns
initial_price = stock_data['Close'].iloc[0]  # Initial price
final_price = stock_data['Close'].iloc[-1]  # Final price

buy_hold_returns = ((final_price - initial_price) / initial_price) * 100

print(f"Buy-and-Hold Returns: {buy_hold_returns:.2f}%")

Buy-and-Hold Returns: 179.02%


In [ ]:
# Compare returns
if total_returns > buy_hold_returns:
    print("The strategy outperformed the benchmark.")
elif total_returns < buy_hold_returns:
    print("The strategy underperformed the benchmark.")
else:
    print("The strategy performed similarly to the benchmark.")


The strategy underperformed the benchmark.
